In [21]:
#1 
# 기존 90개의 컬럼을 줄이려합니다. 컬럼명과 의미, 주의사항은 아래과 같습니다. 
# 파일명 column_pre_v1.xlsx
import pandas as pd
pd.set_option('display.max_rows', None)       
pd.set_option('display.max_columns', None)     
pd.set_option('display.max_colwidth', None)    
pd.set_option('display.width', None)           

df_v1 = pd.read_excel("column_pre_v1.xlsx")
df_v1.head(10)

,column,dtype,missing_rate,desc_ko,ml_drop,ml_note
0,bd,int16,0.000000,원본 나이/출생 관련 값(이상치 가능),삭제 추천,원본 bd는 출생년도/이상치 혼재 → bd_clean 사용 권장
1,bd_clean,float32,0.551209,정제된 나이(출생년도 혼재/이상치 제거 후; 비현실 값은 결측),NaN,결측률 높음: 트리모델은 NaN 유지 가능 / 선형모델은 결측지시자 추가 권장
2,city,int8,0.000000,거주 도시 코드(익명화),NaN,NaN
3,gender,category,0.000000,성별(male/female/unknown),NaN,범주형 처리 필요(원-핫 or CatBoost/LightGBM categorical)
4,is_churn,Int8,0.000000,"이탈 여부(라벨; 1=이탈, 0=유지)",NaN,NaN
5,msno,string,0.000000,유저 ID,삭제 추천,식별자(유저 ID) → 피처로 쓰면 과적합/누수 위험
6,registered_via,int8,0.000000,가입 채널 코드(익명화),NaN,NaN
7,registration_init_time,datetime64[ns],0.000000,가입일,NaN,datetime은 숫자(예: 가입 후 경과일)로 변환 권장
8,registration_month,period[M],0.000000,가입월(년-월),NaN,period는 timestamp/월 인덱스로 변환 권장
9,avg_amount_per_payment,float32,0.000000,결제 1회당 평균 금액,NaN,NaN


In [22]:
#2 
# 삭제에 앞서 data_total의 컬럼을 순서대로 조회했습니다. 
df_v1 = pd.read_csv("data_total.csv")

# 컬럼명 클린업: str로 통일 + 숨은문자/BOM/공백 제거
df_v1.columns = (
    pd.Index(df_v1.columns)
    .map(lambda x: str(x))            # 무조건 문자열화
    .str.replace("\ufeff", "", regex=False)  # BOM 제거
    .str.replace("\u00a0", " ", regex=False) # NBSP(특수 공백) → 일반 공백
    .str.strip()                      # 앞뒤 공백 제거
)

# 컬럼을 순서대로 출력
for i, c in enumerate(df_v1.columns, 1):
    print(i, c)

1 msno
2 city
3 bd
4 gender
5 registered_via
6 registration_init_time
7 bd_clean
8 registration_month
9 is_churn
10 num_days_active_w7
11 total_secs_w7
12 avg_secs_per_day_w7
13 std_secs_w7
14 num_songs_w7
15 avg_songs_per_day_w7
16 num_unq_w7
17 num_25_w7
18 num_100_w7
19 short_play_w7
20 skip_ratio_w7
21 completion_ratio_w7
22 short_play_ratio_w7
23 variety_ratio_w7
24 num_days_active_w14
25 total_secs_w14
26 avg_secs_per_day_w14
27 std_secs_w14
28 num_songs_w14
29 avg_songs_per_day_w14
30 num_unq_w14
31 num_25_w14
32 num_100_w14
33 short_play_w14
34 skip_ratio_w14
35 completion_ratio_w14
36 short_play_ratio_w14
37 variety_ratio_w14
38 num_days_active_w21
39 total_secs_w21
40 avg_secs_per_day_w21
41 std_secs_w21
42 num_songs_w21
43 avg_songs_per_day_w21
44 num_unq_w21
45 num_25_w21
46 num_100_w21
47 short_play_w21
48 skip_ratio_w21
49 completion_ratio_w21
50 short_play_ratio_w21
51 variety_ratio_w21
52 num_days_active_w30
53 total_secs_w30
54 avg_secs_per_day_w30
55 std_secs_w30
56 n

In [23]:
#3 
# 5개 날렸습니다.
drop_cols = ["bd", "registration_init_time", "registration_month", "recency_secs_ratio", "recency_songs_ratio"]  
"""
bd: 결측치가 많아 그냥 둘다 날렸습니다. bd_clean도 날리려했으나 GTP 선생님이 NaN 자체가 신호일 수도 있고 트리 모델은 NaN 잘 처리한다고 냅두랍니다.  
registration_init_time: 가입일 (이미 기반으로 가입 기간 산출했다 생각)
registration_month: 가입 년월 (이미 기반으로 가입 기간 산출했다 생각)
recency_secs_ratio: secs_trend_w7_w30 와 동일
recency_songs_ratio: songs_trend_w7_w30 와 동일
"""
df_v2 = df_v1.drop(columns=drop_cols)

df_v2.head(10)

,msno,city,gender,registered_via,bd_clean,is_churn,num_days_active_w7,total_secs_w7,avg_secs_per_day_w7,std_secs_w7,num_songs_w7,avg_songs_per_day_w7,num_unq_w7,num_25_w7,num_100_w7,short_play_w7,skip_ratio_w7,completion_ratio_w7,short_play_ratio_w7,variety_ratio_w7,num_days_active_w14,total_secs_w14,avg_secs_per_day_w14,std_secs_w14,num_songs_w14,avg_songs_per_day_w14,num_unq_w14,num_25_w14,num_100_w14,short_play_w14,skip_ratio_w14,completion_ratio_w14,short_play_ratio_w14,variety_ratio_w14,num_days_active_w21,total_secs_w21,avg_secs_per_day_w21,std_secs_w21,num_songs_w21,avg_songs_per_day_w21,num_unq_w21,num_25_w21,num_100_w21,short_play_w21,skip_ratio_w21,completion_ratio_w21,short_play_ratio_w21,variety_ratio_w21,num_days_active_w30,total_secs_w30,avg_secs_per_day_w30,std_secs_w30,num_songs_w30,avg_songs_per_day_w30,num_unq_w30,num_25_w30,num_100_w30,short_play_w30,skip_ratio_w30,completion_ratio_w30,short_play_ratio_w30,variety_ratio_w30,secs_trend_w7_w30,secs_trend_w14_w30,days_trend_w7_w14,days_trend_w7_w30,songs_trend_w7_w30,songs_trend_w14_w30,skip_trend_w7_w30,completion_trend_w7_w30,days_since_last_payment,has_ever_paid,days_since_last_cancel,has_ever_cancelled,is_auto_renew_last,last_plan_days,last_payment_method,is_free_user,total_payment_count,total_amount_paid,avg_amount_per_payment,unique_plan_count,subscription_months_est,payment_count_last_30d,payment_count_last_90d
0,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,unknown,7,NaN,0,7,75448.625,10778.3750,9128.5150,338,48.285713,159,39,271,54,0.115385,0.801775,0.159763,0.470414,14,177639.300,12688.5210,10458.7550,842,60.142857,480,127,641,157,0.150831,0.761283,0.186461,0.570071,20,238367.420,11918.3710,9021.4410,1156,57.800000,663,170,863,220,0.147059,0.746540,0.190311,0.573529,30,358554.000,11951.8000,7876.6377,1776,59.200000,1040,277,1296,355,0.155968,0.729730,0.199887,0.585586,0.210425,0.495432,0.500000,0.233333,0.190315,0.474099,-0.040584,0.072045,5.0,1,999.0,0,1,30.0,41.0,0,1,129,129.0,1,1.0,1,1
1,yLkV2gbZ4GLFwqTOXLVHz0VGrMYcgBGgKZ3kj9RiYu8=,4,male,9,30.0,0,6,123668.695,20611.4500,9505.3500,557,92.833336,67,14,518,22,0.025135,0.929982,0.039497,0.120287,6,123668.695,20611.4500,9505.3500,557,92.833336,67,14,518,22,0.025135,0.929982,0.039497,0.120287,6,123668.695,20611.4500,9505.3500,557,92.833336,67,14,518,22,0.025135,0.929982,0.039497,0.120287,6,123668.695,20611.4500,9505.3500,557,92.833336,67,14,518,22,0.025135,0.929982,0.039497,0.120287,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.0,1,999.0,0,1,30.0,39.0,0,2,298,149.0,1,2.0,1,2
2,I0yFvqMoNkM8ZNHb617e1RBzIS/YRKemHO7Wj13EtA0=,13,male,9,63.0,0,3,16989.527,5663.1760,1434.4094,70,23.333334,65,2,65,3,0.028571,0.928571,0.042857,0.928571,10,50269.140,5026.9140,3101.1736,249,24.900000,182,43,195,47,0.172691,0.783133,0.188755,0.730924,15,63667.992,4244.5327,2992.6343,352,23.466667,273,77,206,107,0.218750,0.585227,0.303977,0.775568,18,80453.320,4469.6290,2823.0261,416,23.111110,337,77,269,108,0.185096,0.646635,0.259615,0.810096,0.211172,0.624824,0.300000,0.166667,0.168269,0.598558,-0.156525,0.281937,5.0,1,999.0,0,1,30.0,40.0,0,1,149,149.0,1,1.0,1,1
3,OoDwiKZM+ZGr9P3fRivavgOtglTEaNfWJO4KaJcTTts=,1,unknown,7,NaN,1,1,6168.050,6168.0500,0.0000,23,23.000000,23,0,22,0,0.000000,0.956522,0.000000,1.000000,2,8142.379,4071.1895,2965.4084,35,17.500000,34,2,30,4,0.057143,0.857143,0.114286,0.971429,2,8142.379,4071.1895,2965.4084,35,17.500000,34,2,30,4,0.057143,0.857143,0.114286,0.971429,3,8613.392,2871.1306,2952.4985,38,12.666667,37,3,31,5,0.078947,0.815789,0.131579,0.973684,0.716100,0.945316,0.500000,0.333333,0.605263,0.921053,-0.078947,0.140732,6.0,1,999.0,0,1,30.0,41.0,0,1,149,149.0,1,1.0,1,1
4,4De1jAxNRABoyRBDZ82U0yEmzYkqeOugRGVNIf92Xb8=,4,female,9,28.0,0,2,5703.129,2851.5645,2644.3213,29,14.500000,24,5,24,5,0.172414,0.827586,0.172414,0.827586,5,15160.678,3032.1355,1988.2837,90,18.000000,42,6,82,6,0.066667,0.911111,0.066667,0.466667,8,19365.295,2420.6619,1723.9832,118,14.750000,55,9,105,10,0.07627

In [ ]:
#4
# is_churn: 타겟이므로 맨 마지막열로 뺐습니다.

target = "is_churn"

cols = [c for c in df_v2.columns if c != target] + [target]
df_v3 = df_v2[cols]

# 확인
df_v3.head(10)
df_v3.to_csv("data_total_v3.csv", index=False)

In [25]:
# 컬럼을 순서대로 출력
for i, c in enumerate(df_v3.columns, 1):
    print(i, c)

1 msno
2 city
3 gender
4 registered_via
5 bd_clean
6 num_days_active_w7
7 total_secs_w7
8 avg_secs_per_day_w7
9 std_secs_w7
10 num_songs_w7
11 avg_songs_per_day_w7
12 num_unq_w7
13 num_25_w7
14 num_100_w7
15 short_play_w7
16 skip_ratio_w7
17 completion_ratio_w7
18 short_play_ratio_w7
19 variety_ratio_w7
20 num_days_active_w14
21 total_secs_w14
22 avg_secs_per_day_w14
23 std_secs_w14
24 num_songs_w14
25 avg_songs_per_day_w14
26 num_unq_w14
27 num_25_w14
28 num_100_w14
29 short_play_w14
30 skip_ratio_w14
31 completion_ratio_w14
32 short_play_ratio_w14
33 variety_ratio_w14
34 num_days_active_w21
35 total_secs_w21
36 avg_secs_per_day_w21
37 std_secs_w21
38 num_songs_w21
39 avg_songs_per_day_w21
40 num_unq_w21
41 num_25_w21
42 num_100_w21
43 short_play_w21
44 skip_ratio_w21
45 completion_ratio_w21
46 short_play_ratio_w21
47 variety_ratio_w21
48 num_days_active_w30
49 total_secs_w30
50 avg_secs_per_day_w30
51 std_secs_w30
52 num_songs_w30
53 avg_songs_per_day_w30
54 num_unq_w30
55 num_25_w30